In [ ]:
# dependencies go here
import pandas as pd
from sqlalchemy import create_engine
from config import pw
import os

## set up database

We set up the `minneapolis_housing` database and related tables in Postgresql:

* `neighborhood`
* `home_value`
* `crime`

Code for setting up the tables can be found in the queries.sql file accompanying this notebook.

**Note: Refactor this to work with SQLAlchemy, if time permits**

## load data
### Neighborhood datasets

In [ ]:
# Stacy start

In [ ]:
# Load Minneapolis city dataset for neighborhoods
n_path = os.path.join('.', 'data', 'Minneapolis_Neighborhoods.csv')
neighborhood = pd.read_csv(n_path)
neighborhood_df = neighborhood[['FID', 'BDNAME']].copy()
neighborhood_df.rename(columns={
    'FID': 'id',
    'BDNAME': 'neighborhood'
}, inplace=True)
neighborhood_df.set_index('id', inplace=True)

'''
NOTES:
Noticed there wasn't a codebook that explained what the cryptic column names mean, so used FID based on the fact
that it is a unique ID.

'''

In [ ]:
# Load MNCompass Minneapolis neighborhood dataset 
c_path = os.path.join('.', 'data', 'MSP Neighborhoods_2013-2017.csv')
compass = pd.read_csv(c_path)

In [ ]:
# Stacy end

In [ ]:
# Jenna start

In [ ]:
# Jenna end

In [ ]:
# Katrina start

In [ ]:
# Katrina end

## transforming data

In [ ]:
# Jenna start

In [ ]:
# Jenna end

In [ ]:
# Katrina start

In [ ]:
# Katrina end

## adding to postgres

In [ ]:
# Stacy start

In [ ]:
# Stacy end